In [ ]:
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter

import pandas as pd 

# appid,name_kr,genres,release_date,is_coming_soon,initial_price,description_kr,category,new_description,screenshots

df = pd.read_csv('스팀게임파일.csv')
# 문서화

splitter = RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=50)
doc = []
for index,row in df.iterrows():
    text = (
    f"장르 = {row['genres']}\n"
    f"카테고리 = {row['category']}\n"
    f"간단한설명 = {row['description_kr']}\n"
    f"게임설명 = {row['new_description']}\n"
)
    chunks = splitter.split_text(text)
    for chunk in chunks:
        doc.append(Document(
            page_content=chunk,
            metadata={
                "appid": row['appid'],
                "name": row['name_kr'],
                "genres": row['genres'],
                "release_date": row['release_date'],
                "price": row['initial_price'],
                "screenshots": row['screenshots']   
            }
        ))



In [ ]:

import os
api_key = os.getenv("OPENAI_API_KEY")




embeddings = OpenAIEmbeddings(model= 'text-embedding-3-large',openai_api_key = api_key)

vectorstore = Chroma.from_documents(
    documents=doc,
    embedding=embeddings,
    collection_name='games',
    persist_directory='./games_db'
)